In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!git clone --recursive https://github.com/Abiram929/GazeGaussian.git
%cd GazeGaussian

In [ ]:
!git submodule update --init --recursive

In [ ]:
import torch
import sys
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
!pip install --upgrade pip setuptools wheel ninja

In [ ]:
!pip install fvcore iopath

In [ ]:
!pip install pytorch3d

In [ ]:
try:
    import pytorch3d
    print(f"✓ pytorch3d: {pytorch3d.__version__}")
except ImportError:
    !pip install "git+https://github.com/facebookresearch/pytorch3d.git"
    import pytorch3d
    print(f"✓ pytorch3d from source: {pytorch3d.__version__}")

In [ ]:
!pip install opencv-python h5py tqdm scipy scikit-image lpips kornia

In [ ]:
import os
os.chdir('/content/GazeGaussian/submodules/simple-knn')
with open('simple_knn.cu', 'r') as f:
    content = f.read()
if '#include <cfloat>' not in content:
    content = content.replace('#include <vector>', '#include <vector>\n#include <cfloat>')
    with open('simple_knn.cu', 'w') as f:
        f.write(content)
os.chdir('/content/GazeGaussian')

In [ ]:
%cd submodules/diff-gaussian-rasterization
!python setup.py install
%cd ../..

In [ ]:
%cd submodules/simple-knn
!python setup.py install
%cd ../..

In [ ]:
torch_version = '.'.join(torch.__version__.split('.')[:2])
cuda_version = 'cu' + torch.version.cuda.replace('.', '')
kaolin_url = f"https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-{torch_version}_{cuda_version}.html"
!pip install kaolin -f {kaolin_url}

In [ ]:
try:
    import kaolin
    if hasattr(kaolin, '__version__'):
        print(f"✓ Kaolin: {kaolin.__version__}")
    else:
        raise ImportError("Placeholder package")
except ImportError:
    !pip uninstall -y kaolin
    !git clone --recursive https://github.com/NVIDIAGameWorks/kaolin /content/kaolin
    %cd /content/kaolin
    !pip install -e .
    %cd /content/GazeGaussian
    import kaolin
    print(f"✓ Kaolin from source: {kaolin.__version__}")

In [ ]:
print("\n" + "="*80)
print("VERIFICATION")
print("="*80)

all_good = True

packages = [
    ('torch', 'PyTorch'),
    ('cv2', 'OpenCV'),
    ('h5py', 'h5py'),
    ('pytorch3d', 'PyTorch3D'),
    ('kaolin', 'Kaolin'),
    ('lpips', 'LPIPS'),
    ('kornia', 'Kornia'),
]

for mod, name in packages:
    try:
        m = __import__(mod)
        v = getattr(m, '__version__', 'OK')
        print(f"✓ {name:15s} {v}")
    except ImportError:
        print(f"✗ {name:15s} FAILED")
        all_good = False

try:
    import simple_knn
    print(f"✓ {'simple-knn':15s} OK")
except ImportError:
    print(f"✗ {'simple-knn':15s} FAILED")
    all_good = False

try:
    import diff_gaussian_rasterization
    print(f"✓ {'diff-gauss':15s} OK")
except ImportError:
    print(f"✗ {'diff-gauss':15s} FAILED")
    all_good = False

print("="*80)

if all_good:
    print("✅ ALL PACKAGES INSTALLED!")
else:
    print("⚠ Some packages failed (check above)")

In [ ]:
!mkdir -p data
!mkdir -p /content/drive/MyDrive/GazeGaussian_checkpoints
print("\n✅ Setup complete! Ready for training.")